In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

In [2]:
titanic = pd.read_csv(r"C:\Users\PAUL\Desktop\ML_Projects\dataset\Titanicdata.csv")

In [4]:
titanic.shape

(891, 12)